# Решающие деревья для ДМ

In [1]:
import numpy as np
import pandas as pd
import math

In [8]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
from sklearn.tree import export_graphviz

In [2]:
column_names = ["Targ1","Targ2","Targ3","Targ4","Don","Apgar","Sex","Vospal","Ves",
                "IMT","Somat","Ob_somat","Ob_gin","Ob_inf_ber","Ob_inf_lac","ZMG_lac",
                "Prer_ber","Gestoz","AB_ber","AB_lac","Prob_lac","Rody","7_St aureus","7_Klebsiella","7_E.Coli",
                "7_S.epidermidis","7_Enterococcus spp.","7_S.salivarius","7_K.pneumoniae","7_S. haemolyticus/hominis",
                "7_S. lugdunensis","7_A.pittii","7_S. agalactiae","7_S. anginosus","7_A. baumannii","7_Kocuria kristinae",
                "7_Rothia spp.","7_Corynebacterium spp.","7_Enterobacter spp.","7_A.junii","7_S.maltophilia","7_R.aquatilis",
                "7_Pseudomonas spp","7_E. faecalis","4_St aureus","4_Klebsiella","4_E.Coli","4_S.epidermidis",
                "4_Enterococcus spp.","4_S.salivarius","4_K.pneumoniae","4_S. haemolyticus/hominis",
                "4_S. lugdunensis ","4_A.pittii","4_S. agalactiae","4_S. anginosus","4_A. baumannii","4_Kocuria kristinae",
                "4_Rothia spp.","4_Corynebacterium spp.","4_Enterobacter spp.","4_A.junii","4_S.maltophilia",
                "4_R.aquatilis","4_Pseudomonas spp","4_E. faecalis","10_St aureus","10_Klebsiella",
                "10_E.Coli","10_S.epidermidis","10_Enterococcus spp.","10_S.salivarius","10_K.pneumoniae",
                "10_S. haemolyticus/hominis","10_S. lugdunensis ","10_A.pittii","10_S. agalactiae",
                "10_S. anginosus","10_A. baumannii","10_Kocuria kristinae","10_Rothia spp.",
                "10_Corynebacterium spp.","10_Enterobacter spp.","10_A.junii","10_S.maltophilia",
                "10_R.aquatilis","10_Pseudomonas spp","10_E. faecalis"]

In [3]:
train = pd.read_csv("DM2_train.csv", sep=";", header=None, engine="python", names=column_names)
test = pd.read_csv("DM3_test.csv", sep=";", header=None, engine="python", names=column_names)

In [5]:
numeric_cols_X = ["Don","Apgar","Sex","Vospal","Ves",
                "IMT","Somat","Ob_somat","Ob_gin","Ob_inf_ber","Ob_inf_lac","ZMG_lac",
                "Prer_ber","Gestoz","AB_ber","AB_lac","Prob_lac","Rody"]
numeric_cols_Y = ["Targ1"]

In [6]:
train_features = train[numeric_cols_X]
train_labels = train[numeric_cols_Y]
test_features = test[numeric_cols_X]
test_labels = test[numeric_cols_Y]

## Простое решение

In [12]:
sal_tree = DecisionTreeClassifier(random_state=17)
sal_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [13]:
sal_tree.fit(train_features.values, train_labels.values)



In [16]:
export_graphviz(sal_tree, feature_names=numeric_cols_X, out_file='sal_tree.dot', filled=True)


## Решение (по сложнее)


In [49]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, f1_score

In [36]:
numeric_cols = ["Targ1","Don","Apgar","Sex","Vospal","Ves",
                "IMT","Somat","Ob_somat","Ob_gin","Ob_inf_ber","Ob_inf_lac","ZMG_lac",
                "Prer_ber","Gestoz","AB_ber","AB_lac","Prob_lac","Rody"]

In [37]:
FD= pd.read_csv("DM_all.csv", sep=";", header=None, engine="python", names=numeric_cols)

In [38]:
y = FD['Targ1']
X_train, X_holdout, y_train, y_holdout = train_test_split(FD[numeric_cols_X].values, y, test_size=0.3, random_state=17) 
#test - выделение обучающей выборки

In [39]:
X_train.shape


(42, 18)

In [40]:
X_holdout.shape

(19, 18)

In [41]:
y_train.shape

(42,)

In [42]:
y_holdout.shape

(19,)

In [34]:
X_holdout

array([[0, 2, 1, 0, 3, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2],
       [0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2],
       [1, 1, 1, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 2, 2, 0, 5, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 2, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 3, 2, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 2],
       [0, 1, 1, 1, 5, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 2, 1, 0, 3, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2],
       [0, 2, 2, 0, 4, 3, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2],
       [0, 2, 2, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2],
       [1, 1, 1, 0, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 6, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 2, 0, 5, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 2, 2, 0, 4, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 2, 1, 5, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [43]:
X_holdout

array([[0, 2, 1, 0, 3, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2],
       [0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2],
       [1, 1, 1, 0, 5, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 2, 2, 0, 5, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 2, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 3, 2, 1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 2],
       [0, 1, 1, 1, 5, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 2, 1, 0, 3, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2],
       [0, 2, 2, 0, 4, 3, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2],
       [0, 2, 2, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2],
       [1, 1, 1, 0, 6, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 6, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 2, 0, 5, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 2, 2, 0, 4, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [1, 1, 2, 1, 5, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [44]:
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [81]:
export_graphviz(tree, feature_names=numeric_cols_X, out_file='des_tree.dot', filled=True)

In [45]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [46]:
len(X_holdout)

19

Качество прогнозов будем проверять с помощью простой метрики – доли правильных ответов. Сделаем прогнозы для отложенной выборки. Дерево решений справилось лучше: доля правильных ответов около 94% против 88% у kNN. Но это мы пока выбирали параметры наугад.

In [52]:
from sklearn.metrics import accuracy_score

tree_pred = tree.predict(X_holdout)
print("accuracy = ", accuracy_score(y_holdout, tree_pred)) 
print("recall = ", recall_score(y_holdout, tree_pred)) 
print("precision = ", precision_score(y_holdout, tree_pred)) 
print("roc_auc = ", roc_auc_score(y_holdout, tree_pred)) 
print("f1 = ", f1_score(y_holdout, tree_pred)) 

accuracy =  0.5789473684210527
recall =  0.8333333333333334
precision =  0.4166666666666667
roc_auc =  0.6474358974358975
f1 =  0.5555555555555556


In [53]:
knn_pred = knn.predict(X_holdout)
print("accuracy = ", accuracy_score(y_holdout, knn_pred))
print("recall = ", recall_score(y_holdout, knn_pred)) 
print("precision = ", precision_score(y_holdout, knn_pred)) 
print("roc_auc = ", roc_auc_score(y_holdout, knn_pred)) 
print("f1 = ", f1_score(y_holdout, knn_pred)) 

accuracy =  0.2631578947368421
recall =  0.6666666666666666
precision =  0.25
roc_auc =  0.37179487179487175
f1 =  0.36363636363636365


# Доделать

In [65]:
importances = knn_pred.feature_importances_
indices = np.argsort(importances)[::-1]

AttributeError: 'numpy.ndarray' object has no attribute 'feature_importances_'

In [ ]:
print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))

Теперь настроим параметры дерева на кросс-валидации. Настраивать будем максимальную глубину и максимальное используемое на каждом разбиении число признаков. Суть того, как работает GridSearchCV: для каждой уникальной пары значений параметров max_depth и max_features будет проведена 5-кратная кросс-валидация и выберется лучшее сочетание параметров.

In [54]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [55]:
tree_params = {'max_depth': range(1,11),'max_features': range(4,19)}

In [56]:
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
#GridSearchCV занимается подбором оптимальных параметров для модели из списка tree_params

In [57]:
tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:    6.5s finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 11), 'max_features': range(4, 19)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [58]:
tree_grid.best_params_

{'max_depth': 5, 'max_features': 14}

In [60]:
tree_grid.best_score_

0.8333333333333334

In [63]:
print("accuracy = ", accuracy_score(y_holdout, tree_grid.predict(X_holdout)))
print("recall = ", recall_score(y_holdout, tree_grid.predict(X_holdout)))
print("precision = ", precision_score(y_holdout, tree_grid.predict(X_holdout)))
print("roc_auc = ", roc_auc_score(y_holdout, tree_grid.predict(X_holdout)))
print("f1 = ", f1_score(y_holdout, tree_grid.predict(X_holdout)))

accuracy =  0.47368421052631576
recall =  0.8333333333333334
precision =  0.35714285714285715
roc_auc =  0.5705128205128206
f1 =  0.5


Теперь настроим число соседей в алгоритме kNN.

In [66]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler # Стандартизирует данные

In [67]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

In [68]:
knn_params = {'knn__n_neighbors': range(1, 10)}

In [69]:
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)

In [70]:
knn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    8.4s finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'knn__n_neighbors': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [71]:
knn_grid.best_params_, knn_grid.best_score_

({'knn__n_neighbors': 1}, 0.6666666666666666)

In [72]:
print ("accuracy = ", accuracy_score(y_holdout, knn_grid.predict(X_holdout)))
print ("recall = ", recall_score(y_holdout, knn_grid.predict(X_holdout)))
print ("precision = ", precision_score(y_holdout, knn_grid.predict(X_holdout)))
print ("roc_auc = ", roc_auc_score(y_holdout, knn_grid.predict(X_holdout)))
print ("f1 = ", f1_score(y_holdout, knn_grid.predict(X_holdout)))

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


accuracy =  0.5263157894736842
recall =  0.8333333333333334


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


precision =  0.38461538461538464
roc_auc =  0.608974358974359


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


f1 =  0.5263157894736842


В этом примере дерево показало себя лучше, чем метод ближайших соседей: 94.2% правильных ответов на кросс-валидации и 94.6% на отложенной выборке против 88.6% / 89% для kNN. Более того, в данной задаче дерево проявляет себя очень хорошо, и даже случайный лес (который пока представляем просто как кучу деревьев, которые вместе работают почему-то намного лучше, чем одно дерево) в этом примере показывает долю правильных ответов не намного выше (95.1% на кросс-валидации и 95.3% –на отложенной выборке), а обучается намного дольше.

# RandomForestClassifier

In [73]:
from sklearn.ensemble import RandomForestClassifier

In [74]:
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=17)
print(np.mean(cross_val_score(forest, X_train, y_train, cv=5))) 

0.7388888888888889


In [75]:
forest_params = {'max_depth': range(1,11), 'max_features': range(4,19)}

In [76]:
forest_grid = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, verbose=True)

In [77]:
forest_grid.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   52.2s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  1.5min finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 11), 'max_features': range(4, 19)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [78]:
forest_grid.best_params_, forest_grid.best_score_ 

({'max_depth': 7, 'max_features': 17}, 0.7857142857142857)

In [80]:
print("accuracy = ", accuracy_score(y_holdout, forest_grid.predict(X_holdout))) 
print("recall = ", recall_score(y_holdout, forest_grid.predict(X_holdout))) 
print("precision = ", precision_score(y_holdout, forest_grid.predict(X_holdout))) 
print("roc_auc = ", roc_auc_score(y_holdout, forest_grid.predict(X_holdout))) 
print("f1 = ", f1_score(y_holdout, forest_grid.predict(X_holdout))) 

accuracy =  0.42105263157894735
recall =  0.8333333333333334
precision =  0.3333333333333333
roc_auc =  0.532051282051282
f1 =  0.47619047619047616
